In [22]:
import pandas as pd
import base64
import requests
import hashlib
import hmac
import datetime
from urllib import parse
import urllib.parse
from privateconfig import p_api_key, p_secret_key

In [15]:
def post_huobi(url, data, api_key, secret_key):
    timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
    param = {"AccessKeyId": api_key,
             "SignatureVersion": "2",
             "SignatureMethod": "HmacSHA256",
             "Timestamp": timestamp
            }

    host = urllib.parse.urlparse(url).hostname
    path = urllib.parse.urlparse(url).path
    
    method = "POST"
    
    keys = sorted(param.keys())
    qs0 = '&'.join(['%s=%s' % (key, parse.quote(param[key], safe='')) for key in keys])
    payload0 = '%s\n%s\n%s\n%s' % (method, host, path, qs0)
    dig = hmac.new(secret_key.encode('utf-8'), msg=payload0.encode('utf-8'), digestmod=hashlib.sha256).digest()
    s = base64.b64encode(dig).decode()
    param["Signature"] =  s
    response = requests.post(url + "?" + urllib.parse.urlencode(param) , headers = {'Content-Type': 'application/json'}, json = data)
    return response.json()

In [16]:
data = {'type': 1,
        'trade_type': 0,
        'create_date': 30,
        'status': 0,
        'symbol': 'ETH'
       }

In [17]:
response = post_huobi('https://api.hbdm.com/api/v1/contract_hisorders', data, p_api_key, p_secret_key)

In [23]:
pd.DataFrame(response['data']['orders'])

,order_id,contract_code,symbol,lever_rate,direction,offset,volume,price,create_date,order_source,...,trade_turnover,fee,trade_avg_price,status,order_id_str,fee_asset,liquidation_type,update_time,is_tpsl,real_profit
0,810225244551110656,ETH210326,ETH,20,sell,close,509.0,1850.704,1613214351558,risk,...,5090.0,0.000000,1850.704,6,810225244551110656,ETH,3,1613214351000,0,-0.130644
1,810166248821739520,ETH210326,ETH,20,buy,open,509.0,1943.000,1613200285879,web,...,5090.0,-0.000524,1943.000,6,810166248821739520,ETH,0,1613200317000,0,0.000000
2,810055967001411584,ETH210326,ETH,20,sell,close,560.0,1948.000,1613173992643,web,...,5600.0,-0.000575,1948.000,6,810055967001411584,ETH,0,1613174001000,0,-0.011632
3,810016856600989696,ETH210326,ETH,20,sell,close,560.0,1970.000,1613164667997,web,...,0.0,0.000000,0.000,7,810016856600989696,ETH,0,1613165168000,0,0.000000
4,810010639862214656,ETH210326,ETH,20,buy,open,560.0,1956.537,1613163185811,ios,...,5600.0,-0.001145,1955.914,6,810010639862214656,ETH,0,1613163186000,0,0.000000
5,810009740054044672,ETH210326,ETH,10,sell,close,260.0,1951.000,1613162971280,ios,...,2600.0,-0.000532,1954.049,6,810009740054044672,ETH,0,1613162971000,0,0.010068
6,809985754058620928,ETH210326,ETH,10,buy,open,260.0,1939.374,1613157252572,ios,...,2600.0,-0.000268,1939.374,6,809985754058620928,ETH,0,1613157260000,0,0.000000
7,809984637132017664,ETH210326,ETH,5,sell,close,127.0,1939.187,1613156986277,ios,...,1270.0,-0.000131,1939.187,6,809984637132017664,ETH,0,1613157183000,0,0.002684
8,809976911890997248,ETH210326,ETH,5,buy,open,37.0,1931.139,1613155144435,web,...,370.0,-0.000047,1931.129,6,809976911890997248,ETH,0,1613155157000,0,0.000000
9,809966212812779520,ETH210326,ETH,5,buy,open,27.0,1954.533,1613152593577,ios,...,270.0,-0.000028,1954.533,6,809966212812779520,ETH,0,1613152600000,0,0.000000


In [9]:
data = {'volume': 1,
        'direction': 'buy',
        'offset': 'open',
        'lever_rate': 3,
        'symbol': 'ETH',
        'order_price_type': 'opponent',
        'contract_type': 'next_week'
       }

In [10]:
post_huobi('https://api.hbdm.com/api/v1/contract_order', data, p_api_key, p_secret_key)

{"status":"error","err_code":1047,"err_msg":"Insufficient margin available.","ts":1613339309140}


In [20]:
def get_huobi(url, api_key, secret_key):
    timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
    param = {"AccessKeyId": api_key,
             "SignatureVersion": "2",
             "SignatureMethod": "HmacSHA256",
             "Timestamp": timestamp
            }

    host = urllib.parse.urlparse(url).hostname
    path = urllib.parse.urlparse(url).path
    
    method = "GET"
    
    keys = sorted(param.keys())
    qs0 = '&'.join(['%s=%s' % (key, parse.quote(param[key], safe='')) for key in keys])
    payload0 = '%s\n%s\n%s\n%s' % (method, host, path, qs0)
    dig = hmac.new(secret_key.encode('utf-8'), msg=payload0.encode('utf-8'), digestmod=hashlib.sha256).digest()
    s = base64.b64encode(dig).decode()
    param["Signature"] =  s
    print(urllib.parse.urlencode(param))
    response = requests.get(url + "?" + urllib.parse.urlencode(param) , headers = {'Content-Type': 'application/json'})
    print(response.text)

In [21]:
get_huobi('https://api.hbdm.com/api/v1/contract_open_interest', p_api_key, p_secret_key)

AccessKeyId=8e454fd1-a0241651-cdgs9k03f3-41c25&SignatureVersion=2&SignatureMethod=HmacSHA256&Timestamp=2021-02-14T14%3A39%3A53&symbol=ETH&Signature=B6v1tzztRmMSd1iQ%2FBiPB5Ml1a25Gl03h6HTd2PbmZ0%3D
{"status":"ok","data":[{"volume":1130369.000000000000000000,"amount":15158.515701375487964983,"symbol":"BCH","contract_type":"next_quarter","contract_code":"BCH210625","trade_amount":138070.6564951371445943418259704270905814102,"trade_volume":9968706,"trade_turnover":99687060.000000000000000000},{"volume":342007.000000000000000000,"amount":654183.244070390206579954,"symbol":"EOS","contract_type":"next_week","contract_code":"EOS210226","trade_amount":10554546.0578045196132533917095429281826016334,"trade_volume":5673756,"trade_turnover":56737560.000000000000000000},{"volume":107904.000000000000000000,"amount":38534.528014170467004024,"symbol":"DOT","contract_type":"this_week","contract_code":"DOT210219","trade_amount":86969.1481291308937779156732408279465425856,"trade_volume":249254,"trade_turn